# Estimating Workplace Location

This notebook illustrates how to re-estimate ActivitySim's auto ownership model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - Update the ActivitySim model coefficients and re-run the model in simulation mode.
  
The basic estimation workflow is shown below and explained in the next steps.

![estimation workflow](https://github.com/RSGInc/activitysim/raw/develop/docs/images/estimation_example.jpg)

# Load libraries

In [1]:
import larch  # !conda install larch #for estimation
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import os

# Required Inputs

In addition to a working ActivitySim model setup, estimation mode requires an ActivitySim format household travel survey.  An ActivitySim format household travel survey is very similar to ActivitySim's simulation model tables:

 - households
 - persons
 - tours
 - joint_tour_participants
 - trips (not yet implemented)

Examples of the ActivitySim format household travel survey are included in the [example_estimation data folders](https://github.com/RSGInc/activitysim/tree/develop/activitysim/examples/example_estimation).  The user is responsible for formatting their household travel survey into the appropriate format.  

After creating an ActivitySim format household travel survey, the `scripts/infer.py` script is run to append additional calculated fields.  An example of an additional calculated field is the `household:joint_tour_frequency`, which is calculated based on the `tours` and `joint_tour_participants` tables.  

The input survey files are below.

### Survey households

In [2]:
pd.read_csv("../data_sf/survey_data/override_households.csv")

,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,joint_tour_frequency
0,841891,126,48000,1,4,1,1,0_tours
1,990869,134,48000,2,1,2,2,0_tours
2,125886,113,25900,1,4,1,1,0_tours
3,727893,8,26100,2,1,0,1,0_tours
4,2741769,150,121600,4,1,2,1,0_tours
...,...,...,...,...,...,...,...,...
1995,663493,110,19180,1,6,1,1,0_tours
1996,569375,20,7400,1,6,1,0,0_tours
1997,1445193,17,75000,1,4,0,1,0_tours
1998,2833455,69,0,1,0,0,0,0_tours


### Survey persons

In [3]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4401,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4402,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4403,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey tours

In [4]:
pd.read_csv("../data_sf/survey_data/override_tours.csv")

,tour_id,survey_tour_id,person_id,household_id,tour_type,tour_category,destination,origin,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency,stop_frequency
0,25820,258200,629,629,school,mandatory,12.0,131.0,12.0,21.0,WALK_HVY,NaN,NaN,NaN,121,NaN,1out_0in
1,52265,522650,1274,1274,school,mandatory,10.0,166.0,10.0,11.0,WALK_LRF,NaN,NaN,NaN,86,NaN,0out_0in
2,1117937,11179370,27266,27266,school,mandatory,12.0,9.0,17.0,18.0,WALK_LRF,NaN,NaN,NaN,163,NaN,0out_0in
3,1148523,11485230,28012,28012,school,mandatory,5.0,10.0,8.0,10.0,WALK_LRF,NaN,NaN,NaN,56,NaN,1out_0in
4,1208547,12085470,29476,29476,school,mandatory,13.0,16.0,8.0,15.0,WALK_LOC,NaN,NaN,NaN,61,NaN,1out_0in
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5773,302923726,3029237260,7388383,2749929,maint,atwork,71.0,36.0,15.0,15.0,DRIVEALONEFREE,3.029237e+09,302923742.0,NaN,145,NaN,0out_0in
5774,302942567,3029425670,7388843,2750003,eat,atwork,87.0,42.0,10.0,10.0,DRIVEALONEFREE,3.029426e+09,302942602.0,NaN,85,NaN,0out_0in
5775,302942627,3029426270,7388844,2750003,maint,atwork,67.0,17.0,14.0,14.0,WALK_LOC,3.029426e+09,302942643.0,NaN,135,NaN,0out_0in
5776,305120465,3051204650,7441962,2758909,maint,atwork,130.0,127.0,12.0,12.0,WALK,3.051205e+09,305120481.0,NaN,112,NaN,0out_1in


### Survey joint tour participants

In [5]:
pd.read_csv("../data_sf/survey_data/survey_joint_tour_participants.csv")

,participant_id,tour_id,household_id,person_id,participant_num
0,14429508701,144295087,1606646,3519392,1
1,14429508702,144295087,1606646,3519393,2
2,28367651801,283676518,2628704,6918939,1
3,28367651804,283676518,2628704,6918942,2
4,10079851903,100798519,1173905,2458502,1
...,...,...,...,...,...
223,16297928102,162979281,1769918,3975105,2
224,16297928103,162979281,1769918,3975106,3
225,16297928104,162979281,1769918,3975107,4
226,26353054902,263530549,2519358,6427575,1


# Example Setup if Needed

To avoid duplication of inputs, especially model settings and expressions, the `example_estimation` depends on the `example`.  The following commands create an example setup for use.  The location of these example setups (i.e. the folders) are important because the paths are referenced in this notebook.  The commands below download the skims.omx for the SF county example from the [activitysim resources repository](https://github.com/RSGInc/activitysim_resources).

In [6]:
#!activitysim create -e example_estimation_sf -d test

# Run the Estimation Example

The next step is to run the model with an `estimation.yaml` settings file with the following settings in order to output the EDB for all submodels:

```
enable=True

bundles:
  - school_location
  - workplace_location
  - auto_ownership
  - free_parking
  - cdap
  - mandatory_tour_frequency
  - mandatory_tour_scheduling
  - joint_tour_frequency
  - joint_tour_composition
  - joint_tour_participation
  - joint_tour_destination
  - joint_tour_scheduling
  - non_mandatory_tour_frequency
  - non_mandatory_tour_destination
  - non_mandatory_tour_scheduling
  - tour_mode_choice
  - atwork_subtour_frequency
  - atwork_subtour_destination
  - atwork_subtour_scheduling
  - atwork_subtour_mode_choice
survey_tables:
  households:
    file_name: survey_data/override_households.csv
    index_col: household_id
  persons:
    file_name:  survey_data/override_persons.csv
    index_col: person_id
  tours:
    file_name:  survey_data/override_tours.csv
  joint_tour_participants:
    file_name:  survey_data/override_joint_tour_participants.csv
```

This enables the estimation mode functionality, identifies which models to run and their output estimation data bundles (EDBs), and the input survey tables, which include the override settings for each model choice.  

With this setup, the model will output an EBD with the following tables for this submodel:
  - model settings - workplace_location_model_settings.yaml
  - coefficients - workplace_location_coefficients.csv
  - utilities specification - workplace_location_SPEC.csv
  - land use data - workplace_location_landuse.csv
  - size terms - workplace_location_size_terms.csv
  - alternatives values - workplace_location_alternatives_combined.csv
  - chooser data - workplace_location_choosers_combined.csv
  
The following code runs the software in estimation mode, inheriting the settings from the simulation setup and using the San Francisco county data setup.  It produces the EDB for all submodels but runs all the model steps identified in the inherited settings file.  

In [7]:
%cd test

~/activitysim/activitysim/examples/example_estimation/notebooks/test


In [8]:
#!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

# Load data and prep model for estimation

In [9]:
from activitysim.estimation.larch.location_choice import location_choice_model
model, data = location_choice_model(model_selector="workplace", return_data=True)

# Review data loaded from EDB

Next we can review what was read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

## coefficients

In [10]:
data.coefficients

,value,constrain
coefficient_name,,
coef_dist_0_1,-0.8428,F
coef_dist_1_2,-0.3104,F
coef_dist_2_5,-0.3783,F
coef_dist_5_15,-0.1285,F
coef_dist_15_up,-0.0917,F
coef_dist_0_5_high,0.1500,F
coef_dist_5_up_high,0.0200,F
coef_mode_logsum,0.3000,F


## alt_values

In [11]:
data.alt_values

,person_id,variable,1,2,3,4,5,6,7,8,...,181,182,183,184,185,186,187,188,189,190
0,72241,mode_choice_logsum,-0.6000073826104492,-0.4656491845430101,-0.41794791684660004,-0.4545880924383482,-0.3870339464573669,-0.713189384898449,-0.6002723120707204,-0.7303789868909905,...,-1.2551998198820722,-1.444611856370077,-1.309689434695158,-1.2118895631639268,-1.116133290115392,-1.1401586021041086,-1.268242046936546,-1.1992258811031282,-1.332212687016352,-1.4322381548918224
1,72241,pick_count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,72241,prob,0.044801462203924856,0.06764474840448732,0.004636496646133522,0.038560793011622845,0.029432761447063308,0.007082161657578104,0.021002192072313054,0.007080174169833742,...,9.835213648420107e-05,9.37955508970241e-05,0.0003916917023644097,0.0004549197781990045,0.0006986771541699331,0.0005700269910735453,0.00022288388655805036,0.0015168772236387644,6.456615866696308e-05,0.00038493516540732525
3,72241,shadow_price_size_term_adjustment,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,72241,shadow_price_utility_adjustment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49053,7515185,util_mode_logsum,5.209319664605113,5.263859224685696,5.250578793547065,5.240693588870104,5.266707024418452,5.385882663451519,5.429836866161502,5.413580929225948,...,3.5638723444203606,3.8737395301473363,3.939960434175001,3.896122982249229,4.037021365960745,4.008376112238483,4.022965354450543,4.2561835681622435,4.137475938698699,3.803872808728852
49054,7515185,util_no_attractions,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49055,7515185,util_sample_of_corrections_factor,3.773690429573432,3.2901835133378077,5.885037102917519,3.829120168967722,3.995759873960041,5.287496147764242,4.186480380588901,5.144701771913909,...,8.673308209076758,8.543415039654008,7.12433751865688,7.0286614320788905,6.59573872948536,6.749128801712912,7.618771571501497,5.665033450001899,8.80633209328273,7.046647564016155
49056,7515185,util_size_variable,8.607993215742676,9.068802152955321,6.44746749765505,8.457988473558888,8.245952810520611,6.957999623413992,7.998487266140552,6.960822948235117,...,4.315499470829833,4.3335976552693785,5.721835205763653,5.789241258107118,6.1502039682537495,6.018658905769173,5.06677615336607,6.994814238173049,3.948605626489833,5.759690106594497


## chooser_data

In [12]:
data.chooser_data

,person_id,model_choice,override_choice,income_segment,home_zone_id
0,72241,13,9,1,17
1,72441,100,100,1,60
2,72528,139,126,1,69
3,73144,77,77,1,125
4,73493,117,117,1,133
...,...,...,...,...,...
2577,7514214,80,71,1,56
2578,7514284,187,186,1,72
2579,7514404,105,106,1,81
2580,7514777,87,87,1,106


## landuse

In [13]:
data.landuse

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
zone_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,932.83514,0,0.00000,0.00000,0.00000,3,5.89564,2.875000,1707.375000,2.870167
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,885.61682,0,0.00000,0.00000,0.00000,1,5.84871,5.195214,1631.374751,5.178722
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,716.27252,0,0.00000,0.00000,0.00000,1,5.53231,80.470405,736.891913,72.547987
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,314.00000,0,0.00000,0.00000,0.00000,2,5.64330,7.947368,1180.736842,7.894233
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,314.01431,0,0.00000,72.14684,0.00000,1,5.52555,38.187500,978.875000,36.753679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,4,4,1,2779,8062,376.0,172.0,15.00000,1760,1178,...,0.00000,3,0.00000,0.00000,0.00000,1,2.04173,14.860963,9.411765,5.762347
187,4,4,1,1492,4139,214.0,116.0,10.00000,808,603,...,0.00000,3,0.00000,0.00000,0.00000,2,1.73676,11.841270,6.412698,4.159890
188,4,4,1,753,4072,232.0,11.0,178.00000,4502,1117,...,0.00000,2,3961.04761,17397.79102,11152.93652,1,2.28992,3.984127,23.820106,3.413233


## spec

In [14]:
data.spec

,Label,Description,Expression,coefficient
0,local_dist,NaN,_DIST@skims['DIST'],1
1,util_dist_0_1,"Distance, piecewise linear from 0 to 1 miles","@_DIST.clip(0,1)",coef_dist_0_1
2,util_dist_1_2,"Distance, piecewise linear from 1 to 2 miles","@(_DIST-1).clip(0,1)",coef_dist_1_2
3,util_dist_2_5,"Distance, piecewise linear from 2 to 5 miles","@(_DIST-2).clip(0,3)",coef_dist_2_5
4,util_dist_5_15,"Distance, piecewise linear from 5 to 15 miles","@(_DIST-5).clip(0,10)",coef_dist_5_15
5,util_dist_15_up,"Distance, piecewise linear for 15+ miles",@(_DIST-15.0).clip(0),coef_dist_15_up
6,util_dist_0_5_high,"Distance 0 to 5 mi, high and very high income",@(df['income_segment']>=WORK_HIGH_SEGMENT_ID) ...,coef_dist_0_5_high
7,util_dist_15_up_high,"Distance 5+ mi, high and very high income",@(df['income_segment']>=WORK_HIGH_SEGMENT_ID) ...,coef_dist_5_up_high
8,util_no_attractions,No attractions,@df['size_term']==0,-999
9,util_mode_logsum,Mode choice logsum,mode_choice_logsum,coef_mode_logsum


## size_spec

In [15]:
data.size_spec

,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN
segment,,,,,,
work_low,0.129,0.193,0.383,0.120,0.010,0.164
work_med,0.120,0.197,0.325,0.139,0.008,0.210
work_high,0.110,0.207,0.284,0.154,0.006,0.239
work_veryhigh,0.093,0.270,0.241,0.146,0.004,0.246


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [16]:
model.estimate(method='BHHH', options={'maxiter':1000})

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
coef_dist_0_1,-1.084778,0.0,0.0,NaN,NaN,0,,-1.084778
coef_dist_0_5_high,0.120981,0.0,0.0,NaN,NaN,0,,0.120981
coef_dist_15_up,-0.091700,0.0,0.0,NaN,NaN,0,,-0.091700
coef_dist_1_2,-0.252580,0.0,0.0,NaN,NaN,0,,-0.252580
coef_dist_2_5,-0.369689,0.0,0.0,NaN,NaN,0,,-0.369689
coef_dist_5_15,-0.152259,0.0,0.0,NaN,NaN,0,,-0.152259
coef_dist_5_up_high,-0.015225,0.0,0.0,NaN,NaN,0,,-0.015225
coef_mode_logsum,0.155426,0.0,0.0,NaN,NaN,0,,0.155426


~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.4285556620743595e-13 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.0548159698654926e-14 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.5843413622015934e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 9.271478878467056e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 5.40041590382088e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.744770600289603e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/

~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.3427936898983306e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.0918566447292222e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.080761394517682e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.154190703124784e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.6585359933226885e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 6.110535763038718e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch

~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.6732173561485437e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 7.7892543655461435e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.982429262219377e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.3499717887949852e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 9.16899425347944e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.0865642352374081e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch

...

~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.686642536760436e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.2040518968146315e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.0859458895039555e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.908927636812767e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.4841654184955348e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.3309479799825619e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larc

~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.4981367038826627e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.8450832191167517e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.041221891465534e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 6.801696332508827e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.65535463525798e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.9034131731077365e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/

~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 8.55578258373803e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.2203707908866183e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.0353316917768002e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.6352989090148225e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.7152858458833888e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.5163235104618512e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larc

<ipython-input-16-ce7e9581522b>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate(method='BHHH', options={'maxiter':1000})
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.3212918388814632e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣          loglike: -13521.35475538419
┣                x: -999                    -999.000000
┃                   1                          1.000000
┃                   coef_dist_0_1             -1.084778
┃                   coef_dist_0_5_high         0.120981
┃                   coef_dist_15_up           -0.091700
┃                   coef_dist_1_2             -0.252580
┃                   coef_dist_2_5             -0.369689
┃                   coef_dist_5_15            -0.152259
┃                   coef_dist_5_up_high       -0.015225
┃                   coef_mode_logsum           0.155426
┃                   work_high_AGREMPN        -14.042030
┃                   work_high_FPSEMPN         -2.021234
┃                   work_high_HEREMPN         -1.578781
┃                   work_high_MWTEMPN         -2.074250
┃                   work_high_OTHEMPN         -2.145558
┃                   work_high_RETEMPN         -2.207275
┃                   work_low_AGREMPN         -40.734930
┃                   work_low_FPSEMPN          -0.670108
┃                   work_low_HEREMPN           0.087905
┃                   work_low_MWTEMPN          -0.373028
┃                   work_low_OTHEMPN          -0.705270
┃                   work_low_RETEMPN          -2.047943
┃                   work_med_AGREMPN         -14.886841
┃                   work_med_FPSEMPN          -1.942050
┃                   work_med_HEREMPN          -1.382809
┃                   work_med_MWTEMPN          -0.910436
┃                   work_med_OTHEMPN          -2.535326
┃                   work_med_RETEMPN          -2.120264
┃                   work_veryhigh_AGREMPN     -0.277725
┃                   work_veryhigh_FPSEMPN     -1.757306
┃                   work_veryhigh_HEREMPN     -1.621894
┃                   work_veryhigh_MWTEMPN     -1.722069
┃                   work_veryhigh_OTHEMPN     -2.483268
┃                   work_veryhigh_RETEMPN     -2.375156
┃                   dtype: float64
┣        tolerance: 9.957944723983296e-06
┣            steps: array([0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
┃                          0.015625, 0.015625, 0.015625, 0.015625,

### Estimated coefficients

In [17]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
-999,-999.,NA,NA,,-999.00,fixed value
1,1.00,NA,NA,,1.00,fixed value
coef_dist_0_1,-1.08,0.270,-4.02,***,0.00,
coef_dist_0_5_high,0.121,0.0339,3.57,***,0.00,
coef_dist_15_up,-0.0917,0.00145,-63.43,***,0.00,
coef_dist_1_2,-0.253,0.115,-2.20,*,0.00,
coef_dist_2_5,-0.370,0.0329,-11.23,***,0.00,
coef_dist_5_15,-0.152,0.0419,-3.63,***,0.00,
coef_dist_5_up_high,-0.0152,0.0576,-0.26,,0.00,
coef_mode_logsum,0.155,0.0455,3.41,***,0.00,


# Output Estimation Results

In [18]:
from activitysim.estimation.larch import update_coefficients, update_size_spec
result_dir = data.edb_directory/"estimated"

## Write updated utility coefficients

In [19]:
update_coefficients(
    model, data, result_dir,
    output_file="workplace_location_coefficients_revised.csv",
);

## Write updated size coefficients

In [20]:
update_size_spec(
    model, data, result_dir, 
    output_file="workplace_location_size_terms.csv",
)

,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,work_low,workplace,0.000000,0.044251,0.175515,0.374554,0.169451,6.988475e-19,0.236229,0.000,0.000000,0.000000,0.000000
1,work_med,workplace,0.000000,0.120498,0.144005,0.251914,0.079565,3.439741e-07,0.404018,0.000,0.000000,0.000000,0.000000
2,work_high,workplace,0.000000,0.159104,0.191636,0.298286,0.169233,1.153219e-06,0.181741,0.000,0.000000,0.000000,0.000000
3,work_veryhigh,workplace,0.000000,0.062723,0.116348,0.133219,0.056296,5.108942e-01,0.120520,0.000,0.000000,0.000000,0.000000
4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.592000,0.408000
5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000,0.000000,0.000000,0.000000
6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,1.000000,0.000000,0.000000
7,escort,non_mandatory,0.000000,0.225000,0.000000,0.144000,0.000000,0.000000e+00,0.000000,0.465,0.166000,0.000000,0.000000
8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.000000,0.000000
9,eatout,non_mandatory,0.000000,0.742000,0.000000,0.258000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.000000,0.000000


## Write the model estimation report, including coefficient t-statistic and log likelihood

In [21]:
model.to_xlsx(
    result_dir/"workplace_location_model_estimation.xlsx", 
    data_statistics=False,
);

# Next Steps

The final step is to either manually or automatically copy the `workplace_location_coefficients_revised.csv` file and `workplace_location_size_terms.csv` file to the configs folder, rename them to `workplace_location_coefficients.csv` and `destination_choice_size_terms.csv`, and run ActivitySim in simulation mode.

In [22]:
pd.read_csv(result_dir/"workplace_location_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_dist_0_1,-1.084778,F
1,coef_dist_1_2,-0.252580,F
2,coef_dist_2_5,-0.369689,F
3,coef_dist_5_15,-0.152259,F
4,coef_dist_15_up,-0.091700,F
5,coef_dist_0_5_high,0.120981,F
6,coef_dist_5_up_high,-0.015225,F
7,coef_mode_logsum,0.155426,F


In [23]:
pd.read_csv(result_dir/"workplace_location_size_terms.csv")

,index,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,0,work_low,workplace,0.000000,0.044251,0.175515,0.374554,0.169451,6.988475e-19,0.236229,0.000,0.000000,0.000000,0.000000
1,1,work_med,workplace,0.000000,0.120498,0.144005,0.251914,0.079565,3.439741e-07,0.404018,0.000,0.000000,0.000000,0.000000
2,2,work_high,workplace,0.000000,0.159104,0.191636,0.298286,0.169233,1.153219e-06,0.181741,0.000,0.000000,0.000000,0.000000
3,3,work_veryhigh,workplace,0.000000,0.062723,0.116348,0.133219,0.056296,5.108942e-01,0.120520,0.000,0.000000,0.000000,0.000000
4,4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.592000,0.408000
5,5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000,0.000000,0.000000,0.000000
6,6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,1.000000,0.000000,0.000000
7,7,escort,non_mandatory,0.000000,0.225000,0.000000,0.144000,0.000000,0.000000e+00,0.000000,0.465,0.166000,0.000000,0.000000
8,8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.000000,0.000000
9,9,eatout,non_mandatory,0.000000,0.742000,0.000000,0.258000,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.000000,0.000000
